In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/ibm-hr-analytics-attrition-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv


In [2]:
df=pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [4]:
df = df.drop(['Over18','EmployeeCount','EmployeeNumber','StandardHours'],axis=1)

In [5]:
df = df.drop(['PerformanceRating','HourlyRate','MonthlyRate'],axis=1)

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Attrition'] = label_encoder.fit_transform(df['Attrition'])
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['OverTime'] = label_encoder.fit_transform(df['OverTime'])
df['MaritalStatus'] = label_encoder.fit_transform(df['MaritalStatus'])

In [11]:
def get_unique_values(df, column_name):
    unique_values = df[column_name].unique()
    print(f"Unique values in '{column_name}': {unique_values}")
    return unique_values

# Call the function for 'BusinessTravel'
unique_business_travel = get_unique_values(df, 'BusinessTravel')
unique_business_travel = get_unique_values(df, 'Department')
unique_business_travel = get_unique_values(df, 'EducationField')
unique_business_travel = get_unique_values(df, 'JobRole')

Unique values in 'BusinessTravel': ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
Unique values in 'Department': ['Sales' 'Research & Development' 'Human Resources']
Unique values in 'EducationField': ['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']
Unique values in 'JobRole': ['Sales Executive' 'Research Scientist' 'Laboratory Technician'
 'Manufacturing Director' 'Healthcare Representative' 'Manager'
 'Sales Representative' 'Research Director' 'Human Resources']


In [12]:
df = pd.get_dummies(df, columns=['BusinessTravel'], drop_first=True)
df= pd.get_dummies(df, columns=['Department'], drop_first=True)
df = pd.get_dummies(df, columns=['EducationField'], drop_first=True)
df = pd.get_dummies(df, columns=['JobRole'], drop_first=True)
#df = pd.get_dummies(df, columns=['MaritalStatus'], drop_first=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 41 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Age                                1470 non-null   int64
 1   Attrition                          1470 non-null   int64
 2   DailyRate                          1470 non-null   int64
 3   DistanceFromHome                   1470 non-null   int64
 4   Education                          1470 non-null   int64
 5   EnvironmentSatisfaction            1470 non-null   int64
 6   Gender                             1470 non-null   int64
 7   JobInvolvement                     1470 non-null   int64
 8   JobLevel                           1470 non-null   int64
 9   JobSatisfaction                    1470 non-null   int64
 10  MaritalStatus                      1470 non-null   int64
 11  MonthlyIncome                      1470 non-null   int64
 12  NumCompaniesWorked  

In [14]:
corr_matrix = df.corr()
attrition_corr = corr_matrix['Attrition'].sort_values(ascending=False)
print(attrition_corr)

Attrition                            1.000000
OverTime                             0.246118
MaritalStatus                        0.162070
JobRole_Sales Representative         0.157234
BusinessTravel_Travel_Frequently     0.115143
JobRole_Laboratory Technician        0.098290
Department_Sales                     0.080855
DistanceFromHome                     0.077924
EducationField_Technical Degree      0.069355
EducationField_Marketing             0.055781
NumCompaniesWorked                   0.043494
JobRole_Human Resources              0.036215
Gender                               0.029453
JobRole_Sales Executive              0.019774
JobRole_Research Scientist          -0.000360
PercentSalaryHike                   -0.013478
EducationField_Other                -0.017898
Education                           -0.031373
EducationField_Life Sciences        -0.032703
YearsSinceLastPromotion             -0.033019
RelationshipSatisfaction            -0.045872
EducationField_Medical            

In [15]:
df = df.drop(['NumCompaniesWorked','Gender','PercentSalaryHike','Education'],axis=1)

In [16]:
X  = df.drop('Attrition',axis=1)
y = df['Attrition'].astype(int)

In [18]:
import joblib
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X=X_train,y=y_train)
test_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score, confusion_matrix
accuracyrf = accuracy_score(y_test,test_pred)
print('Accuracy score is ' + str((accuracyrf * 100 )) )
joblib.dump(classifier, 'eapmp_rf.pkl')  # You can change the file path if needed

print("Model saved successfully!")

Accuracy score is 84.35374149659864
Model saved successfully!


In [22]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  # Importing the SVC model
from sklearn.metrics import accuracy_score, confusion_matrix

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40)

# Initialize the SVC model
classifier = SVC()

# Fit the model to the training data
classifier.fit(X_train, y_train)

# Make predictions on the test set
test_pred = classifier.predict(X_test)

# Calculate accuracy
accuracysvc = accuracy_score(y_test, test_pred)
print('Accuracy score is ' + str((accuracysvc * 100)))

# Save the trained model
joblib.dump(classifier, 'eapmp_svc.pkl')  # You can change the file path if needed

print("Model saved successfully!")

Accuracy score is 84.6938775510204
Model saved successfully!


In [28]:
import joblib
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression  # Importing the Logistic Regression model
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)



classifier = LGBMClassifier()

# Fit the model to the training data
classifier.fit(X_train, y_train)

# Make predictions on the test set
test_pred = classifier.predict(X_test)

# Calculate accuracy
accuracylgb = accuracy_score(y_test, test_pred)
print('Accuracy score is ' + str((accuracy * 100)))

# Save the trained model
joblib.dump(classifier, 'eapmp_LGB.pkl')  # You can change the file path if needed

print("Model saved successfully!")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 165, number of negative: 864
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000205 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 780
[LightGBM] [Info] Number of data points in the train set: 1029, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.160350 -> initscore=-1.655627
[LightGBM] [Info] Start training from score -1.655627
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Accuracy score is 85.71428571428571
Model saved successfully!


In [27]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, Flatten, Dropout, concatenate
from tensorflow.keras.utils import to_categorical

# Assuming df is the pre-processed DataFrame
X = df.drop('Attrition', axis=1)
y = df['Attrition'].astype(int)

# One-hot encode the target if it's categorical
y_categorical = to_categorical(y)

# Scale the features for CNN-ANN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape data for CNN input
X_cnn = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_categorical, test_size=0.30, random_state=42)

# -------------------- Step 1: Train XGBoost --------------------
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

# XGBoost predictions (probabilities)
xgb_train_preds = xgb.predict_proba(X_train)
xgb_test_preds = xgb.predict_proba(X_test)

# -------------------- Step 2: Train CNN-ANN --------------------
# Define CNN-ANN model
input_layer = Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))

# CNN branch
cnn_layer = Conv1D(filters=32, kernel_size=3, activation='relu')(input_layer)
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
cnn_layer = Flatten()(cnn_layer)

# ANN branch
ann_layer = Dense(128, activation='relu')(cnn_layer)
ann_layer = Dropout(0.5)(ann_layer)
ann_layer = Dense(64, activation='relu')(ann_layer)

# Output layer
output_layer = Dense(y_train_cnn.shape[1], activation='softmax')(ann_layer)

# Compile the model
cnn_ann_model = Model(inputs=input_layer, outputs=output_layer)
cnn_ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN-ANN model
cnn_ann_model.fit(X_train_cnn, y_train_cnn, epochs=5, batch_size=32, validation_split=0.2)

# CNN-ANN predictions (probabilities)
cnn_train_preds = cnn_ann_model.predict(X_train_cnn)
cnn_test_preds = cnn_ann_model.predict(X_test_cnn)

# -------------------- Step 3: Combine Predictions --------------------
# Concatenate XGBoost and CNN-ANN predictions
train_combined = np.hstack((xgb_train_preds, cnn_train_preds))
test_combined = np.hstack((xgb_test_preds, cnn_test_preds))

# -------------------- Step 4: Train Meta-Classifier --------------------
# Use a simple ANN as the meta-classifier
meta_input = Input(shape=(train_combined.shape[1],))
meta_layer = Dense(64, activation='relu')(meta_input)
meta_layer = Dropout(0.3)(meta_layer)
meta_output = Dense(y_train_cnn.shape[1], activation='softmax')(meta_layer)

meta_model = Model(inputs=meta_input, outputs=meta_output)
meta_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the meta-classifier
meta_model.fit(train_combined, y_train_cnn, epochs=10, batch_size=64, validation_split=0.2)

# -------------------- Step 5: Evaluate the Ensemble --------------------
# Evaluate on the test set
final_test_preds = meta_model.predict(test_combined)
final_test_accuracy = accuracy_score(y_test, np.argmax(final_test_preds, axis=1))

print(f"Final Ensemble Test Accuracy: {final_test_accuracy * 100:.2f}%")
# Save the meta-classifier ANN model
meta_model.save('meta_classifiereapmp.h5')
print("Meta-classifier model saved as meta_classifiereap.h5")


Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8207 - loss: 0.5238 - val_accuracy: 0.7913 - val_loss: 0.4719
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8246 - loss: 0.4290 - val_accuracy: 0.7913 - val_loss: 0.4365
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8350 - loss: 0.3603 - val_accuracy: 0.8301 - val_loss: 0.4191
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8927 - loss: 0.2855 - val_accuracy: 0.8010 - val_loss: 0.3997
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8960 - loss: 0.2694 - val_accuracy: 0.8204 - val_loss: 0.3954
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4020 - loss: 0.7144 - val_accuracy: 1.0000 - val_loss: 0.6081
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8748 - loss: 0.5856 - val_accuracy: 1.0000 - val_loss: 0.5026
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [33]:
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your trained model (either from VotingClassifier, StackingClassifier, or individual model)
model = joblib.load('/kaggle/working/eapmp_LGB.pkl') 


# Assuming you have a dataset `X` (features) ready to predict on
# Example: Load or prepare your dataset
# X = pd.read_csv('your_data.csv')  # Your data goes here

# Predict probabilities of attrition (assuming binary classification: 0 = No attrition, 1 = Attrition)
attrition_prob = model.predict_proba(X)[:, 1]  # Get the probability for class '1' (attrition)
att=attrition_prob*100
# Add the predicted probability as a new column to the dataset
df['attrition_probability'] = att

# Optionally, save the updated dataset with the new column
#X.to_csv('data_with_attrition_probability.csv', index=False)

# Print the first few rows to verify
df.head()

,Age,Attrition,DailyRate,DistanceFromHome,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MaritalStatus,MonthlyIncome,...,EducationField_Technical Degree,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,attrition_probability
0,41,1,1102,1,2,3,2,4,2,5993,...,False,False,False,False,False,False,False,True,False,94.763151
1,49,0,279,8,3,2,2,2,1,5130,...,False,False,False,False,False,False,True,False,False,0.160272
2,37,1,1373,2,4,2,1,3,2,2090,...,False,False,True,False,False,False,False,False,False,95.656485
3,33,0,1392,3,4,3,1,3,1,2909,...,False,False,False,False,False,False,True,False,False,1.563660
4,27,0,591,2,1,3,1,2,1,3468,...,False,False,True,False,False,False,False,False,False,4.863055


In [36]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Age                                1470 non-null   int64
 1   DailyRate                          1470 non-null   int64
 2   DistanceFromHome                   1470 non-null   int64
 3   EnvironmentSatisfaction            1470 non-null   int64
 4   JobInvolvement                     1470 non-null   int64
 5   JobLevel                           1470 non-null   int64
 6   JobSatisfaction                    1470 non-null   int64
 7   MaritalStatus                      1470 non-null   int64
 8   MonthlyIncome                      1470 non-null   int64
 9   OverTime                           1470 non-null   int64
 10  RelationshipSatisfaction           1470 non-null   int64
 11  StockOptionLevel                   1470 non-null   int64
 12  TotalWorkingYears   

In [46]:
import pandas as pd
import joblib

# Function to load the model
def load_model(model_path):
    """
    Loads the trained model from the specified path.
    """
    try:
        model = joblib.load('/kaggle/working/eapmp_LGB.pkl')
        #print("Model loaded successfully!")
        return model
    except FileNotFoundError:
        print(f"Error: Model file not found at {model_path}")
        return None

# Function to preprocess input data
def preprocess_input(input_data, feature_columns):
    """
    Converts user input into a DataFrame and ensures it matches the training features.
    """
    input_df = pd.DataFrame([input_data])
    input_df = input_df[feature_columns]  # Reorder columns to match training data
    return input_df

# Function to make predictions
def predict_attrition(model, input_df):
    """
    Predicts attrition probability and class using the trained model.
    """
    # Predict probability of attrition
    #attrition_probability = model.predict_proba(input_df)[:, 1]
    # Predict the class (0 or 1)
    attrition_prediction = model.predict(input_df)
    
    # Format results
    #probability = round(attrition_probability[0] * 100, 2)
    prediction = "Attrition" if attrition_prediction[0] == 1 else "No Attrition"
    return  prediction

# Main script
if __name__ == "__main__":
    # Path to the trained model
    model_path = '/kaggle/working/eap_LGB.pkl'  # Replace with the actual model path
    
    # Load the model
    model = load_model(model_path)
    
    if model:
        # Example input data from the user (35 attributes only)
        input_data = {
            'Age': 25,
            'DailyRate': 1102,
            'DistanceFromHome': 1,
            'EnvironmentSatisfaction': 2,
            'JobInvolvement': 3,
            'JobLevel': 2,
            'JobSatisfaction': 4,
            'MaritalStatus': 2,
            'MonthlyIncome': 5000,
            'OverTime': 1,
            'RelationshipSatisfaction': 1,
            'StockOptionLevel': 2,
            'TotalWorkingYears': 5,
            'TrainingTimesLastYear': 1,
            'WorkLifeBalance': 1,
            'YearsAtCompany': 5,
            'YearsInCurrentRole': 4,
            'YearsSinceLastPromotion': 1,
            'YearsWithCurrManager': 3,
            'BusinessTravel_Travel_Frequently': 0,
            'BusinessTravel_Travel_Rarely': 1,
            'Department_Research & Development': 0,
            'Department_Sales': 1,
            'EducationField_Life Sciences': 0,
            'EducationField_Marketing': 1,
            'EducationField_Medical': 0,
            'EducationField_Other': 0,
            'EducationField_Technical Degree': 0,
            'JobRole_Human Resources': 0,
            'JobRole_Laboratory Technician': 0,
            'JobRole_Manager': 0,
            'JobRole_Manufacturing Director': 0,
            'JobRole_Research Director': 0,
            'JobRole_Research Scientist': 0,
            'JobRole_Sales Executive': 1,
            'JobRole_Sales Representative': 0
        }

        # Feature columns (35 attributes only)
        feature_columns = [
            'Age', 'DailyRate', 'DistanceFromHome', 'EnvironmentSatisfaction',
            'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MaritalStatus',
            'MonthlyIncome', 'OverTime', 'RelationshipSatisfaction', 'StockOptionLevel',
            'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 
            'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 
            'YearsWithCurrManager', 'BusinessTravel_Travel_Frequently', 
            'BusinessTravel_Travel_Rarely', 'Department_Research & Development', 
            'Department_Sales', 'EducationField_Life Sciences', 
            'EducationField_Marketing', 'EducationField_Medical', 
            'EducationField_Other', 'EducationField_Technical Degree', 
            'JobRole_Human Resources', 'JobRole_Laboratory Technician', 
            'JobRole_Manager', 'JobRole_Manufacturing Director', 
            'JobRole_Research Director', 'JobRole_Research Scientist', 
            'JobRole_Sales Executive', 'JobRole_Sales Representative'
        ]

        # Preprocess input data
        input_df = preprocess_input(input_data, feature_columns)
        
        # Make predictions
        prediction = predict_attrition(model, input_df)
        
        # Display results
        #print(f"Attrition Probability: {probability}%")
        print(f"Attrition Prediction: {prediction}")


Attrition Prediction: No Attrition
